<a href="https://colab.research.google.com/github/MateusSilva12/aprendizado1/blob/main/4_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random as rndm
import time

In [ ]:
def make_gene(initial=None):
    if initial is None:
        initial = [0] * 16
    mapp = {}
    gene = list(range(1, 17))
    rndm.shuffle(gene)
    for i in range(16):
        mapp[gene[i]] = i
    for i in range(16):
        if initial[i] != 0 and gene[i] != initial[i]:
            temp = gene[i], gene[mapp[initial[i]]]
            gene[mapp[initial[i]]], gene[i] = temp
            mapp[initial[i]], mapp[temp[0]] = i, mapp[initial[i]]
    return gene

In [ ]:
def make_chromosome(initial=None):
    if initial is None:
        initial = [[0] * 16] * 16
    chromosome = []
    for i in range(16):
        chromosome.append(make_gene(initial[i]))
    return chromosome

In [ ]:
def make_population(count, initial=None):
    if initial is None:
        initial = [[0] * 16] * 16
    population = []
    for _ in range(count):
        population.append(make_chromosome(initial))
    return population

In [ ]:
def get_fitness(chromosome):
    """Calculate the fitness of a chromosome (puzzle)."""
    fitness = 0
    for i in range(16): # For each column
        seen = {}
        for j in range(16): # Check each cell in the column
            if chromosome[j][i] in seen:
                seen[chromosome[j][i]] += 1
            else:
                seen[chromosome[j][i]] = 1
        for key in seen: # Subtract fitness for repeated numbers
            fitness -= (seen[key] - 1)
    for m in range(4): # For each 3x3 square
        for n in range(4):
            seen = {}
            for i in range(4 * n,4 * (n + 1)):  # Check cells in 3x3 square
                for j in range(4 * m, 4 * (m + 1)):
                    if chromosome[j][i] in seen:
                        seen[chromosome[j][i]] += 1
                    else:
                        seen[chromosome[j][i]] = 1
            for key in seen: # Subtract fitness for repeated numbers
                fitness -= (seen[key] - 1)
    return fitness

In [ ]:
ch = make_chromosome()
print(get_fitness(ch))


def pch(ch):
    for i in range(16):
        for j in range(16):
            print(ch[i][j], end=" ")
        print("")

-159


In [ ]:
def crossover(ch1, ch2):
    new_child_1 = []
    new_child_2 = []
    for i in range(16):
        x = rndm.randint(0, 1)
        if x == 1:
            new_child_1.append(ch1[i])
            new_child_2.append(ch2[i])
        elif x == 0:
            new_child_2.append(ch1[i])
            new_child_1.append(ch2[i])
    return new_child_1, new_child_2


In [ ]:
def mutation(ch, pm, initial):
    for i in range(16):
        x = rndm.randint(0, 100)
        if x < pm * 100:
            ch[i] = make_gene(initial[i])
    return ch

In [ ]:

def read_puzzle(address):
    puzzle = []
    f = open(address, 'r')
    for row in f:
        temp = row.split()
        puzzle.append([int(c) for c in temp])
    return puzzle

In [ ]:
def r_get_mating_pool(population):
    fitness_list = []
    pool = []
    for chromosome in population:
        fitness = get_fitness(chromosome)
        fitness_list.append((fitness, chromosome))
    fitness_list.sort()
    weight = list(range(1, len(fitness_list) + 1))
    for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weight)[0]
        pool.append(ch[1])
    return pool

In [ ]:
def w_get_mating_pool(population):
    fitness_list = []
    pool = []
    for chromosome in population:
        fitness = get_fitness(chromosome)
        fitness_list.append((fitness, chromosome))
    weight = [fit[0] - fitness_list[0][0] for fit in fitness_list]
    for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weights=weight)[0]
        pool.append(ch[1])
    return pool

In [ ]:
def get_offsprings(population, initial, pm, pc):
    new_pool = []
    i = 0
    while i < len(population):
        ch1 = population[i]
        ch2 = population[(i + 1) % len(population)]
        x = rndm.randint(0, 100)
        if x < pc * 100:
            ch1, ch2 = crossover(ch1, ch2)
        new_pool.append(mutation(ch1, pm, initial))
        new_pool.append(mutation(ch2, pm, initial))
        i += 2
    return new_pool

In [ ]:
# Population size
POPULATION = 1000

# Number of generations
REPETITION = 1000

# Probability of mutation
PM = 0.1

# Probability of crossover
PC = 0.95

# Main genetic algorithm function
def genetic_algorithm(initial_file):
    initial = read_puzzle(initial_file)
    population = make_population(POPULATION, initial)
    for _ in range(REPETITION):
        mating_pool = r_get_mating_pool(population)
        rndm.shuffle(mating_pool)
        population = get_offsprings(mating_pool, initial, PM, PC)
        fit = [get_fitness(c) for c in population]
        m = max(fit)
        if m == 0:
            return population
    return population

In [ ]:
tic = time.time()
r = genetic_algorithm("/content/4x4-01 (1).txt")

toc = time.time()
print("time_taken: ", toc - tic)
fit = [get_fitness(c) for c in r]
m = max(fit)
print(max(fit))

# Print the chromosome with the highest fitness
for c in r:
    if get_fitness(c) == m:
        pch(c)
        break

time_taken:  477.1574845314026
-65
5 9 14 3 10 2 6 13 15 11 8 4 16 12 7 1 
2 4 6 12 14 9 1 7 10 5 16 13 11 3 15 8 
2 8 6 15 5 11 16 3 7 12 9 10 1 4 14 13 
16 11 2 7 12 8 15 4 3 5 1 14 9 6 13 10 
9 8 13 7 15 14 11 2 12 5 3 6 1 4 10 16 
10 5 12 4 16 1 13 3 6 15 14 11 7 8 2 9 
14 15 3 16 2 10 7 8 13 1 5 4 12 9 6 11 
6 10 9 11 3 15 12 1 16 8 2 7 4 5 13 14 
12 16 11 14 8 7 3 2 4 9 15 6 13 10 1 5 
7 1 6 2 5 12 4 10 8 16 13 3 14 11 9 15 
9 6 8 13 1 15 2 11 14 7 10 5 4 12 16 3 
4 3 5 15 13 16 9 14 11 2 1 12 6 7 8 10 
11 14 10 9 7 6 8 15 1 5 4 2 3 13 16 12 
6 2 7 4 12 13 10 16 3 15 11 9 5 14 1 8 
3 1 16 6 11 5 14 12 10 13 7 8 15 9 2 4 
8 13 12 5 1 3 4 2 9 14 16 6 10 15 11 7 
